In [2]:
import dask.dataframe as dd


In [3]:
ddf = dd.read_csv('/data2st1/junyi/snmc_bedgraph/merged/all_unified.bedGraph',sep='\t',header=0, assume_missing=True)

In [5]:
ddf.columns

Index(['chrom', 'start', 'end', 'ABC_NN.CGN', 'ACB-BST-FS_D1_Gaba.CGN',
       'ADP-MPO_Trp73_Glut.CGN', 'AD_Serpinb7_Glut.CGN',
       'AHN_Onecut3_Gaba.CGN', 'AHN-RCH-LHA_Otp_Fezf1_Glut.CGN',
       'AHN-SBPV-PVHd_Pdrm12_Gaba.CGN',
       ...
       'TH_Prkcd_Grin2c_Glut.CGN', 'TMv-PMv_Tbx3_Hist-Gaba.CGN',
       'TRS-BAC_Sln_Glut.CGN', 'TU-ARH_Otp_Six6_Gaba.CGN',
       'VCO_Mafa_Meis2_Glut.CGN', 'Vip_Gaba.CGN', 'VLMC_NN.CGN',
       'VMH_Fezf1_Glut.CGN', 'VMH_Nr5a1_Glut.CGN', 'ZI_Pax6_Gaba.CGN'],
      dtype='object', length=277)

In [9]:
matrix = ddf.iloc[:,3:]

In [10]:
matrix = matrix.compute()

In [11]:
import scipy.sparse as sp
sparse_matrix = sp.csr_matrix(matrix.values)


In [13]:
import scanpy as sc
adata = sc.AnnData(sparse_matrix)

In [16]:
adata = adata.T

In [22]:
region_df = ddf.iloc[:,0:3]

In [24]:
region_df = region_df.compute()

In [26]:
region_df['start'] = region_df['start'].astype(int)
region_df['end'] = region_df['end'].astype(int)

In [27]:
region_df

,chrom,start,end
0,chr1,3000826,3000827
1,chr1,3000827,3000828
2,chr1,3001006,3001007
3,chr1,3001007,3001008
4,chr1,3001017,3001018
...,...,...,...
76479,chrY,90844638,90844639
76480,chrY,90844648,90844649
76481,chrY,90844649,90844650
76482,chrY,90844675,90844676


In [21]:
adata.obs_names = ddf.columns[3:]

In [29]:
adata.var = region_df

/home/junyichen/anaconda3/envs/snapatac2/lib/python3.10/site-packages/anndata/_core/anndata.py:747: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [32]:
adata.var.index = adata.var['chrom'] + ':' + adata.var['start'].astype(str) + '-' + adata.var['end'].astype(str)

In [35]:
adata.var.index = adata.var.index.astype(str)
adata.obs.index = adata.obs.index.astype(str)

In [36]:
adata.write_h5ad('/data2st1/junyi/snmc_bedgraph/merged/meth_frac_all.h5ad')

In [39]:
mapping = {
    'Oligo_NN': 'OPC-Oligo',
    'Microglia_NN': 'Immune',
    'Endo_NN': 'Vascular',
    'Astro-TE_NN': 'Astro-Epen',
    'Astro-OLF_NN': 'Astro-Epen',
    'Astro-NT_NN': 'Astro-Epen',
    'Astroependymal_NN': 'Astro-Epen',
    'Hypendymal_NN': 'Astro-Epen',
    'Ependymal_NN': 'Astro-Epen',
    'Tanycyte_NN': 'Astro-Epen',
    'Bergmann_NN': 'Astro-Epen',
    'CHOR_NN': 'Astro-Epen',
    'Astro-CB_NN': 'Astro-Epen',
    'OPC_NN': 'OPC-Oligo',
    'OEC_NN': 'Astro-Epen',
    'BAM_NN': 'Immune',
    'DC_NN': 'Immune',
    'Lymphoid_NN': 'Immune',
    'VLMC_NN': 'Vascular',
    'ABC_NN': 'Immune',
    'SMC_NN': 'Vascular',
    'Peri_NN': 'Vascular'
}

In [52]:
adata.obs['Subtype'] = adata.obs.index.str.replace('.CGN','')

In [53]:
adata.obs['Celltype.l1'] = adata.obs['Subtype'].map(mapping).fillna('Neuron')


In [58]:
adata.obs['Celltype.l2'] = adata.obs['Subtype'].str.split('_').str[0]+"_"+adata.obs['Subtype'].str.split('_').str[-1]

In [60]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000)

: 